In [230]:
import numpy as np
import pandas as pd
import itertools
from calc_chemfeat_2 import Perovskite
from matminer.featurizers.base import BaseFeaturizer


import pymatgen as mg
from pymatgen.core.periodic_table import _pt_data

from flatten_dict import flatten

## attempt to assign sites and guess oxidation states

In [11]:
cation_site={'Ba':'A','Co':'B','Fe':'B','Zr':'B','Y':'B'}

for k,v in cation_site.items():
    print(k, mg.Element(k).data['Shannon radii'],'\n',mg.Element(k).X)

Ba {'2': {'VI': {'': {'crystal_radius': 1.49, 'ionic_radius': 1.35}}, 'VII': {'': {'crystal_radius': 1.52, 'ionic_radius': 1.38}}, 'VIII': {'': {'crystal_radius': 1.56, 'ionic_radius': 1.42}}, 'IX': {'': {'crystal_radius': 1.61, 'ionic_radius': 1.47}}, 'X': {'': {'crystal_radius': 1.66, 'ionic_radius': 1.52}}, 'XI': {'': {'crystal_radius': 1.71, 'ionic_radius': 1.57}}, 'XII': {'': {'crystal_radius': 1.75, 'ionic_radius': 1.61}}}} 
 0.89
Co {'2': {'IV': {'High Spin': {'crystal_radius': 0.72, 'ionic_radius': 0.58}}, 'V': {'': {'crystal_radius': 0.81, 'ionic_radius': 0.67}}, 'VI': {'Low Spin': {'crystal_radius': 0.79, 'ionic_radius': 0.65}, 'High Spin': {'crystal_radius': 0.885, 'ionic_radius': 0.745}}, 'VIII': {'': {'crystal_radius': 1.04, 'ionic_radius': 0.9}}}, '3': {'VI': {'High Spin': {'crystal_radius': 0.75, 'ionic_radius': 0.61}, 'Low Spin': {'crystal_radius': 0.685, 'ionic_radius': 0.545}}}, '4': {'IV': {'': {'crystal_radius': 0.54, 'ionic_radius': 0.4}}, 'VI': {'High Spin': {'cry

In [28]:
help(mg.periodic_table)

Help on module pymatgen.core.periodic_table in pymatgen.core:

NAME
    pymatgen.core.periodic_table

DESCRIPTION
    # coding: utf-8
    # Copyright (c) Pymatgen Development Team.
    # Distributed under the terms of the MIT License.

CLASSES
    enum.Enum(builtins.object)
        Element
    monty.json.MSONable(builtins.object)
        Specie
            DummySpecie
    
    class DummySpecie(Specie)
     |  A special specie for representing non-traditional elements or species. For
     |  example, representation of vacancies (charged or otherwise), or special
     |  sites, etc.
     |  
     |  Args:
     |      symbol (str): An assigned symbol for the dummy specie. Strict
     |          rules are applied to the choice of the symbol. The dummy
     |          symbol cannot have any part of first two letters that will
     |          constitute an Element symbol. Otherwise, a composition may
     |          be parsed wrongly. E.g., "X" is fine, but "Vac" is not
     |          beca

In [317]:
class PerovskiteStructurePredict():
    def __init__(self,formula, force_anions=[],force_cations=[]):
        self.formula = formula
        self.force_anions = force_anions
        self.force_cations = force_cations
        
        self.composition = mg.Composition(formula)
        self.el_amt_dict = self.composition.get_el_amt_dict()
        self._set_allowed_ox_states()
        self._set_common_ox_states()
        
    def _set_allowed_ox_states(self):
        self._allowed_ox_states = {}
        for el in self.composition.elements:
            ox_states = [int(k) for k in el.data['Shannon radii'].keys()]
            
            if np.sign(min(ox_states)) != np.sign(max(ox_states)):
                if el.symbol in self.force_anions:
                    ox_states = [ox for ox in ox_states if ox < 0]
                elif el.symbol in self.force_cations:
                    ox_states = [ox for ox in ox_states if ox > 0]
                else:
                    print(f'Warning: both positive and negative oxidation states allowed for {el}')
            self._allowed_ox_states[el.symbol] = ox_states
            
    def _get_allowed_ox_states(self):
        return self._allowed_ox_states
    
    allowed_ox_states = property(_get_allowed_ox_states,_set_allowed_ox_states)
    
    def _set_common_ox_states(self):
        self._common_ox_states = {}
        for el in self.composition.elements:
            self._common_ox_states[el.symbol] = el.data['Common oxidation states']
            
    def _get_common_ox_states(self):
        return self._common_ox_states
    
    common_ox_states = property(_get_common_ox_states,_set_common_ox_states)
    
    @property
    def anions(self):
        return [el for el,ox in self.allowed_ox_states.items() if ox[0] < 0]
    
    @property
    def cations(self):
        return [el for el,ox in self.allowed_ox_states.items() if ox[0] > 0]
    
    @property
    def max_Asite_radii(self):
        """
        Get maximum A-site (12-coordinated) radius (across common oxidation states) for each cation
        """
        max_A_rad = {}
        for cat in self.cations:
            fd = flatten(mg.Element(cat).data['Shannon radii'])
            A_rad = [rad for keys,rad in fd.items() if int(keys[0]) in self.common_ox_states[cat] 
                                            and keys[1]=='XII' and keys[3]=='ionic_radius']
            #if no 12-coordinated radius, allow any CN
            if len(A_rad) == 0:
                A_rad = [rad for keys,rad in fd.items() if int(keys[0]) in self.common_ox_states[cat] 
                                            and keys[3]=='ionic_radius']
            max_A_rad[cat] = max(A_rad)
        return max_A_rad
    
    @property
    def multivalents(self):
        return [el for el,ox in self.allowed_ox_states.items() if len(ox) > 1]
    
    def fill_sites(self,site_tol_frac=0.1):
        """
        Assign cations to A and B sites. Fill A-site with largest cations first
        
        Parameters:
        -----------
        site_tol_frac: fractional tolerance for site over/under-occupancy
        """
        
        #get total anions to determine total A-site occupancy
        tot_anions = sum([self.el_amt_dict[an] for an in self.anions])
        tot_cations = sum([self.el_amt_dict[cat] for cat in self.cations])
        A_tot = tot_anions/3
        site_tol = site_tol_frac*A_tot #site tolerance
        if tot_cations > 2*(A_tot + site_tol) or tot_cations < 2*(A_tot - site_tol):
            raise Exception("Formula is not site balanced")
        
        #sort cations from largest to smallest
        rad_sort = sorted(psp.max_Asite_radii.items(),key=lambda x: x[1], reverse=True)
        
        Asite = {}
        Bsite = {}
        A_rem = A_tot #remaining A-site units to fill
        
        #fill in A-site with largest cations 
        #while A_rem > 0:
        for i, (el, rad) in enumerate(rad_sort):
            #print(el,A_rem)
            amt = self.el_amt_dict[el]
            if round(A_rem - amt,4) >= 0: #if enough space, put all of el on A-site
                Asite[el] = amt
                A_rem -= amt
                #print(1)
            elif amt <= A_rem + site_tol and len(Bsite)==0: #if space within site tolerance, assign el to site for which it matches size better
                rA = np.average([r for e,r in rad_sort[:i]], weights=[self.el_amt_dict[e] for e,r in rad_sort[:i]])
                rB = np.average([r for e,r in rad_sort[i+1:]], weights=[self.el_amt_dict[e] for e,r in rad_sort[i+1:]])
                if abs(rA - rad) < abs(rB-rad):
                    Asite[el] = amt
                else:
                    Bsite[el] = amt
                A_rem = 0
            elif A_rem <= site_tol: #if remaining space is below tolerance, consider A-site full
                Bsite[el] = amt
                A_rem = 0
                #print(2)
            elif A_rem > 0: #if remaining space above tolerance, split el between A and B sites
                Asite[el] = A_rem
                Bsite[el] = amt - A_rem
                A_rem = 0
                print(f"Warning: {el} split between A- and B-sites")
                #print(3)
#             else: #if no space remaining, assign to B-site
#                 Bsite[el] = amt
#                 print(4)
                    
        self.Asite_amt = Asite
        self.Bsite_amt = Bsite
        
    def ox_combos(self):
        multi_ox = [self.allowed_ox_states[el] for el in self.multivalents]
        fixed_ox = {el:ox[0] for el,ox in self.allowed_ox_states.items() if len(ox)==1}
        #fixed_radii = 
        for tup in itertools.product(*multi_ox):
            calculate = True
            ox_states = fixed_ox.copy()
            for el, ox in zip(self.multivalents, tup):
                ox_states[el] = ox
            #print(ox_states)
            A_radii = {}
            for el, amt in self.Asite_amt.items():
            #for el,ox in ox_states.items():
                ox = ox_states[el]
                srd = flatten(mg.Element(el).data['Shannon radii'])
                rad = [r for keys, r in srd.items() if int(keys[0])==ox and keys[1]=='XII' 
                               and keys[2] in ('','High Spin') and keys[3]=='ionic_radius']
                if len(rad)>1:
                    raise Exception(f"Found multiple A-site radii for {el} with oxidation state {ox}")
                elif len(rad)==0:
                    print(f"Warning: no A-site radii found for {el} with oxidation state {ox}")
                    calculate=False
                    break
                else:
                    A_radii[el] = rad[0]
                
            B_radii = {}
            for el, amt in self.Bsite_amt.items():
                ox = ox_states[el]
                srd = flatten(mg.Element(el).data['Shannon radii'])
                rad = [r for keys, r in srd.items() if int(keys[0])==ox and keys[1]=='VI' 
                               and keys[2] in ('','High Spin') and keys[3]=='ionic_radius']   
                
                if len(rad)>1:
                    raise Exception(f"Found multiple B-site radii for {el} with oxidation state {ox}")
                elif len(rad)==0:
                    print(f"Warning: no B-site radii found for {el} with oxidation state {ox}")
                    calculate=False
                    break
                else:
                    B_radii[el] = rad[0]
                    
            X_radii = {}
            for el in self.anions:
                amt = self.el_amt_dict[el]
                ox = ox_states[el]
                X_radii[el] = mg.Element(el).data['Shannon radii'][f'{ox}']['VI']['']['ionic_radius']
                
            #print(ox_states,A_radii,B_radii)
#             print(A_radii, self.Asite_amt)
            #print(B_radii, self.Bsite_amt)
            if calculate:
                rA = np.average(list(A_radii.values()), weights=list(self.Asite_amt.values()))
                rB = np.average(list(B_radii.values()), weights=list(self.Bsite_amt.values()))
                rX = np.average(list(X_radii.values()), weights=[self.el_amt_dict[el] for el in self.anions])
                nA = np.average([ox for el,ox in ox_states.items() if el in self.Asite_amt.keys()], 
                                weights=list(self.Asite_amt.values()))
                nB = np.average([ox for el,ox in ox_states.items() if el in self.Bsite_amt.keys()], 
                                weights=list(self.Bsite_amt.values()))
                nX = np.average([ox for el,ox in ox_states.items() if el in self.anions], 
                                weights=[self.el_amt_dict[el] for el in self.anions])
                t = (rA+rX)/((2**0.5)*(rB+rX)) #Goldschmidt
                tau = (rX/rB)-nA*(nA-(rA/rB)/np.log(rA/rB)) #Bartel
                tot_cat_charge = sum([ox*self.el_amt_dict[el] for el,ox in ox_states.items() if el in self.cations])
                delta = 3 + tot_cat_charge/nX
                print({k:v for k,v in ox_states.items() if k in self.multivalents}, t, tau, delta)
                #print(rA,rB, B_radii, t, tau)
                
                    
            
        

In [318]:
psp = PerovskiteStructurePredict('BaCo0.4Fe0.4Y0.1Zr0.1O3')
psp.allowed_ox_states
psp.common_ox_states
psp.cations
psp.max_Asite_radii
psp.fill_sites()
psp.Asite_amt, psp.Bsite_amt
#psp.multivalents
psp.ox_combos()
#psp.anions

Ba 1.0
Y 0.0
Fe 0
Co 0
Zr 0
{'Co': 2, 'Fe': 2} 0.9799223809261086 3.4882423929928184 0.8500000000000001
{'Co': 2, 'Fe': 3} 1.0049062376636015 3.503506804306367 0.6499999999999999
{'Co': 2, 'Fe': 4} 1.016423787665477 3.530865610352526 0.4500000000000002
{'Co': 3, 'Fe': 2} 1.0049062376636015 3.5035068043063666 0.6499999999999999
{'Co': 3, 'Fe': 3} 1.0311973892303818 3.5836031772726216 0.4500000000000002
{'Co': 3, 'Fe': 4} 1.0433291232213275 3.641303722705136 0.25
{'Co': 4, 'Fe': 2} 1.0203218654705215 3.5428822744737527 0.4500000000000002
{'Co': 4, 'Fe': 3} 1.0474367181946398 3.663736939048138 0.25
{'Co': 4, 'Fe': 4} 1.0599558821571256 3.7410926186924005 0.04999999999999982


In [307]:
mg.Element('Co').data['Shannon radii'], mg.Element('Fe').data['Shannon radii']

({'2': {'IV': {'High Spin': {'crystal_radius': 0.72, 'ionic_radius': 0.58}},
   'V': {'': {'crystal_radius': 0.81, 'ionic_radius': 0.67}},
   'VI': {'High Spin': {'crystal_radius': 0.885, 'ionic_radius': 0.745},
    'Low Spin': {'crystal_radius': 0.79, 'ionic_radius': 0.65}},
   'VIII': {'': {'crystal_radius': 1.04, 'ionic_radius': 0.9}}},
  '3': {'VI': {'High Spin': {'crystal_radius': 0.75, 'ionic_radius': 0.61},
    'Low Spin': {'crystal_radius': 0.685, 'ionic_radius': 0.545}}},
  '4': {'IV': {'': {'crystal_radius': 0.54, 'ionic_radius': 0.4}},
   'VI': {'High Spin': {'crystal_radius': 0.67, 'ionic_radius': 0.53}}}},
 {'2': {'IV': {'High Spin': {'crystal_radius': 0.77, 'ionic_radius': 0.63}},
   'IVSQ': {'High Spin': {'crystal_radius': 0.78, 'ionic_radius': 0.64}},
   'VI': {'High Spin': {'crystal_radius': 0.92, 'ionic_radius': 0.78},
    'Low Spin': {'crystal_radius': 0.75, 'ionic_radius': 0.61}},
   'VIII': {'High Spin': {'crystal_radius': 1.06, 'ionic_radius': 0.92}}},
  '3': {'IV

In [96]:
for el,data in _pt_data.items():
    try:
        ox = [int(k) for k in data['Shannon radii'].keys()]
        if np.sign(min(ox)) != np.sign(max(ox)):
            print(el, min(ox),max(ox))
    except KeyError:
        pass

Br -1 7
Cl -1 7
F -1 7
I -1 7
N -3 5
S -2 6
Se -2 6
Te -2 6


In [319]:
mg.Element('Co').data

{'Atomic mass': 58.933195,
 'Atomic no': 27,
 'Atomic orbitals': {'1s': -275.616639,
  '2p': -28.152095,
  '2s': -32.379758,
  '3d': -0.322368,
  '3p': -2.388285,
  '3s': -3.651812,
  '4s': -0.204497},
 'Atomic radius': 1.35,
 'Atomic radius calculated': 1.52,
 'Boiling point': '3200 K',
 'Brinell hardness': '700 MN m<sup>-2</sup>',
 'Bulk modulus': '180 GPa',
 'Coefficient of linear thermal expansion': '13.0 x10<sup>-6</sup>K<sup>-1</sup>',
 'Common oxidation states': [2, 3],
 'Critical temperature': 'no data K',
 'Density of solid': '8900 kg m<sup>-3</sup>',
 'Electrical resistivity': '6 10<sup>-8</sup> &Omega; m',
 'Electronic structure': '[Ar].3d<sup>7</sup>.4s<sup>2</sup>',
 'ICSD oxidation states': [1, 2, 3, 4],
 'IUPAC ordering': 67,
 'Ionic radii': {'2': 0.885, '3': 0.75, '4': 0.67},
 'Ionic radii hs': {'2': 0.885, '3': 0.75, '4': 0.67},
 'Ionic radii ls': {'2': 0.79, '3': 0.685},
 'Liquid range': '1432 K',
 'Melting point': '1768 K',
 'Mendeleev no': 64,
 'Metallic radius': 1.

In [320]:
760.4*1000/(6.022e23*1.602e-19)

7.88204382515877

In [38]:
dir(mg.Element('O'))

['X',
 '__class__',
 '__doc__',
 '__module__',
 'as_dict',
 'average_anionic_radius',
 'average_cationic_radius',
 'average_ionic_radius',
 'block',
 'common_oxidation_states',
 'data',
 'from_Z',
 'from_dict',
 'from_row_and_group',
 'full_electronic_structure',
 'ground_state_term_symbol',
 'group',
 'icsd_oxidation_states',
 'ionic_radii',
 'is_actinoid',
 'is_alkali',
 'is_alkaline',
 'is_chalcogen',
 'is_halogen',
 'is_lanthanoid',
 'is_metalloid',
 'is_noble_gas',
 'is_post_transition_metal',
 'is_quadrupolar',
 'is_rare_earth_metal',
 'is_transition_metal',
 'is_valid_symbol',
 'iupac_ordering',
 'max_oxidation_state',
 'metallic_radius',
 'min_oxidation_state',
 'name',
 'nmr_quadrupole_moment',
 'number',
 'oxidation_states',
 'print_periodic_table',
 'row',
 'term_symbols',
 'valence',
 'value']

In [40]:
mg.Element('O').print_periodic_table()

H                                                                   He 
Li  Be                                          B   C   N   O   F   Ne 
Na  Mg                                          Al  Si  P   S   Cl  Ar 
K   Ca  Sc  Ti  V   Cr  Mn  Fe  Co  Ni  Cu  Zn  Ga  Ge  As  Se  Br  Kr 
Rb  Sr  Y   Zr  Nb  Mo  Tc  Ru  Rh  Pd  Ag  Cd  In  Sn  Sb  Te  I   Xe 
Cs  Ba      Hf  Ta  W   Re  Os  Ir  Pt  Au  Hg  Tl  Pb  Bi  Po  At  Rn 
Fr  Ra                                                                 
        La  Ce  Pr  Nd  Pm  Sm  Eu  Gd  Tb  Dy  Ho  Er  Tm  Yb  Lu     
        Ac  Th  Pa  U   Np  Pu  Am  Cm  Bk  Cf  Es  Fm  Md  No  Lr     


NoneType

In [41]:
x = mg.Element('O')
x.H

Element H

In [62]:
[int(k) for k in x.Fe.data['Shannon radii'].keys()]

[2, 3, 4, 6]

In [59]:
elements = ['Ba','Co','Ce','Fe','Zr','Y','Yb','Mn','In','Ga','Gd','Cu','Ni']

for el in elements:
    data = mg.Element(el).data
    try: 
        cos = data['Common oxidation states']
        sos = data['Shannon radii'].keys()
        print(el, cos, sos)
    except KeyError:
        print(f'No data for {el}')
    

Ba [2] dict_keys(['2'])
Co [2, 3] dict_keys(['2', '3', '4'])
Ce [3, 4] dict_keys(['3', '4'])
Fe [2, 3] dict_keys(['2', '3', '4', '6'])
Zr [4] dict_keys(['4'])
Y [3] dict_keys(['3'])
Yb [3] dict_keys(['2', '3'])
Mn [2, 4, 7] dict_keys(['2', '3', '4', '5', '6', '7'])
In [3] dict_keys(['3'])
Ga [3] dict_keys(['3'])
Gd [3] dict_keys(['3'])
Cu [2] dict_keys(['1', '2', '3'])
Ni [2] dict_keys(['2', '3', '4'])


{'Atomic mass': 227.0, 'Atomic no': 89, 'Atomic orbitals': {'1s': -3443.110367, '2p': -572.7627, '2s': -592.622878, '3d': -119.541743, '3p': -137.654394, '3s': -147.320716, '4d': -23.57061, '4f': -12.278225, '4p': -31.761846, '4s': -36.15826, '5d': -3.222752, '5p': -6.06511, '5s': -7.713078, '6d': -0.137786, '6p': -0.744524, '6s': -1.19698, '7s': -0.126551}, 'Atomic radius': 1.95, 'Atomic radius calculated': 'no data', 'Boiling point': '3573 K', 'Brinell hardness': 'no data MN m<sup>-2</sup>', 'Bulk modulus': 'no data GPa', 'Coefficient of linear thermal expansion': 'no data x10<sup>-6</sup>K<sup>-1</sup>', 'Common oxidation states': [3], 'Critical temperature': 'no data K', 'Density of solid': '10070 kg m<sup>-3</sup>', 'Electrical resistivity': 'no data 10<sup>-8</sup> &Omega; m', 'Electronic structure': '[Rn].6d<sup>1</sup>.7s<sup>2</sup>', 'Ionic radii': {'3': 1.26}, 'Liquid range': '2250 K', 'Melting point': '1323 K', 'Mendeleev no': 48, 'Mineral hardness': 'no data', 'Molar volum